In [8]:
import pandas as pd
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import deplacy
en = spacy.load('en_core_web_sm')
en.add_pipe('spacytextblob')

In [9]:
df = pd.read_csv("scraped_output.csv")

In [10]:
df.dropna(inplace=True)
df.head(3)

,Unnamed: 0,Review,Date,Stars,Restaurant
0,0,"(Translated by Google) Good hamburgers, delici...",5 days ago,4 stars,TFDB
3,3,"Burger was ok, not fantastic. Place was extre...",2 weeks ago,3 stars,TFDB
4,4,"One of the better burgers I've had, the bread ...",2 weeks ago,5 stars,TFDB


In [11]:
import spacy
import deplacy
from spacytextblob.spacytextblob import SpacyTextBlob
en = spacy.load('en_core_web_sm')
en.add_pipe('spacytextblob')


In [22]:
### REFERENCE
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import deplacy
en = spacy.load('en_core_web_sm')
en.add_pipe('spacytextblob')

#Text 1 - With Punctuation and conj at each sentiment split
#Text 2 - With only 1 conjuction
#Text 3 - With no conj so we are only looking at the noun_dict

# text = "In my opinion, the food was really really good , but their ambiance was shit, and my mom thinks the waiter was amazing"
# text = "In my opinion, the food was really really good but their ambiance was shit, my mom thinks the waiter was amazing"
text = "onion rings were good"

doc = en(text)
deplacy.render(doc)

seen = set() # keep track of covered words

chunks = []
dict_noun = {} # This is for the dict of Noun - Adverb, adj, verb

for sent in doc.sents:
    #calculate the dict_noun first

    for word in sent:
            if word.pos_ == 'NOUN':
                current_noun = word.text
                dict_noun[current_noun] = []
            if word.pos_ == 'ADV' or word.pos_ == 'ADJ' or word.pos_ == 'VERB':
                dict_noun[current_noun].append(word.text)

    #See if there are conjunctions to split on. If so get the heads for the subtrees
    heads = [cc for cc in sent if cc.dep_ == 'conj']
    print(heads)

    # If there are no conjunctions split only on punctuation.
    if(len(heads)==0):
        counter = 0
        array = []
        for word in sent:
            if word.pos_ != 'PUNCT':
                array.append(word)
                continue
            chunk = (' '.join([ww.text for ww in array]))
            chunks.append( (counter, chunk) )
            counter = counter + 1
            array = []
        chunk = (' '.join([ww.text for ww in array]))
        chunks.append( (counter, chunk) )
    else:
        for head in heads:
            words = [None]*100
            counter = 0
            for i in head.subtree:
                if(i in heads):
                    if(i != head):
                        break
                if(i.pos_ == 'PUNCT'):
                    break
                words[counter] = i
                counter = counter + 1
                if(i.dep_ == 'cc'):
                    break
            res = []
            for val in words:
                if val != None :
                    res.append(val)
            for word in res:
                seen.add(word)
            chunk = (' '.join([ww.text for ww in res]))
            chunks.append( (head.i, chunk) )
        unseen = [ww for ww in sent if ww not in seen]
        counter = 0
        array = []
        for word in unseen:
            if word.pos_ != 'PUNCT':
                array.append(word)
                continue
            chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))
            chunks.append( (counter, chunk) )
            counter = counter + 1
            array = []
        chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))
        chunks.append( (counter, chunk) )
#         chunk = ' '.join([ww.text for ww in unseen if ww.dep_ != 'cc' and ww.pos_ != 'PUNCT'])
#         chunks.append( (sent.root.i, chunk) )

chunks = sorted(chunks, key=lambda x: x[0])
print(dict_noun)
# for ii, chunk in chunks:
#     if (len(chunk) != 0):
#         print("The chunk is: " + chunk)
#     doc = en(chunk)
#     for i in doc.sents:
#         for cc in i:
#             if cc.dep_ == 'nsubj':
#                 continue
#                 print("Noun :" + cc.text)
#     if(doc._.polarity != 0.0):
#         print("Polarity: " + str(doc._.polarity))

onion NOUN <╗   compound
rings NOUN ═╝<╗ nsubj
were  AUX  ═╗═╝ ROOT
good  ADJ  <╝   acomp
[]
{'onion': [], 'rings': ['good']}


In [12]:
# Function to do feature-descriptor extraction from each row:
def get_feature_descriptors(text):
#     text = row["text"]

    doc = en(text)
#     deplacy.render(doc)
    seen = set() # keep track of covered words
    dict_noun = {} # This is for the dict of Noun - Adverb, adj, verb
    chunks = []
    for sent in doc.sents:
    #calculate the dict_noun first

        for word in sent:
                
                if word.pos_ == 'NOUN':
                    current_noun = word.text
                    dict_noun[current_noun] = []
                if word.pos_ == 'ADV' or word.pos_ == 'ADJ' or word.pos_ == 'VERB':
                    try:
                        dict_noun[current_noun].append(word.text)
                    except:
                        pass;
                    

        #See if there are conjunctions to split on. If so get the heads for the subtrees
        heads = [cc for cc in sent if cc.dep_ == 'conj']
#         print(heads)

        # If there are no conjunctions split only on punctuation.
        if(len(heads)==0):
            counter = 0
            array = []
            for word in sent:
                if word.pos_ != 'PUNCT':
                    array.append(word)
                    continue
                chunk = (' '.join([ww.text for ww in array]))
                chunks.append( (counter, chunk) )
                counter = counter + 1
                array = []
            chunk = (' '.join([ww.text for ww in array]))
            chunks.append( (counter, chunk) )
        else:
            for head in heads:
                words = [None]*100
                counter = 0
                for i in head.subtree:
                    if(i in heads):
                        if(i != head):
                            break
                    if(i.pos_ == 'PUNCT'):
                        break
                    words[counter] = i
                    counter = counter + 1
                    if(i.dep_ == 'cc'):
                        break
                res = []
                for val in words:
                    if val != None :
                        res.append(val)
                for word in res:
                    seen.add(word)
                chunk = (' '.join([ww.text for ww in res]))
                chunks.append( (head.i, chunk) )
            unseen = [ww for ww in sent if ww not in seen]
            counter = 0
            array = []
            for word in unseen:
                if word.pos_ != 'PUNCT':
                    array.append(word)
                    continue
                chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))
                chunks.append( (counter, chunk) )
                counter = counter + 1
                array = []
            chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))
            chunks.append( (counter, chunk) )
    return dict_noun

In [13]:
# trialdf = df.copy()
trialdf = df#.sample(100, random_state=4222)
# trialdf

trialdf["feature_descriptors"] = trialdf["Review"].apply(lambda x:get_feature_descriptors(x))

# for i in tqdm(range(0, len(df))):
    

In [14]:
trialdf.iloc[5, :]["feature_descriptors"]

{'burgers': ['reminds', 'only'],
 'thing': ['missing', 'perfect'],
 'weather': ['Fantastic'],
 'char': [],
 'patty': [],
 'onion': [],
 'rings': ['nicely', 'done']}

In [15]:
trialdf.to_csv("featured_output.csv")

### Extract Features

In [19]:
import benepar, spacy
nlp = spacy.load('en_core_web_sm')
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})
doc = nlp('The time for action is now. It is never too late to do something.')


[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\vibkr\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [6]:
subclauses = []
for sent in list(doc.sents):
    print(sent._.parse_string)
    print(sent._.labels)
    print(list(sent._.children))
    subclauses.append(list(sent._.children))

subclauses = [item for sublist in subclauses for item in sublist]

(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
('S',)
[The time for action, is now, .]
(S (NP (PRP It)) (VP (VBZ is) (ADVP (RB never)) (ADJP (RB too) (JJ late) (S (VP (TO to) (VP (VB do) (NP (NN something))))))) (. .))
('S',)
[It, is never too late to do something, .]


In [7]:
subclauses

[The time for action, is now, ., It, is never too late to do something, .]